# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql import functions as F
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType

import datetime as dt

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [19]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [20]:
user_log.filter(user_log.userId == "").\
groupby("page").count().\
sort(desc("count")).show()

+-----+-----+
| page|count|
+-----+-----+
| Home|  187|
|Login|  126|
|About|   15|
| Help|    8|
+-----+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [21]:
# TODO: use this space to explore the behavior of the user with an empty string
#probably unregistred user

#see pages
user_log.filter(user_log.userId == "").\
select("sessionId").dropDuplicates().\
count()
#186 unregistered user sessions

186

In [22]:
user_log.registerTempTable("dful")

In [30]:
user_log = spark.sql("SELECT *, CAST(ts / 1000.0 AS TIMESTAMP) as ts2 FROM dful")
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- ts2: timestamp (nullable = true)



In [33]:
tsdf = user_log.filter(user_log.userId == "").\
groupby("sessionId").\
agg(F.min("ts2").alias("MinTS"),
    F.max("ts2").alias("MaxTS"))
tsdf.printSchema()

root
 |-- sessionId: long (nullable = true)
 |-- MinTS: timestamp (nullable = true)
 |-- MaxTS: timestamp (nullable = true)



In [39]:
#challenge for later: How long do unregsitred user spend time on the webiste?

Column<b'ts2'>





# Question 3

How many female users do we have in the data set?

In [40]:
user_log.filter(user_log.userId != '').\
groupby("gender").count().\
sort(desc("count")).show()

+------+-----+
|gender|count|
+------+-----+
|     M| 5844|
|     F| 3820|
+------+-----+



# Question 4

How many songs were played from the most played artist?

In [41]:
user_log.filter(user_log.page == "NextSong").\
groupby("artist").count().\
sort(desc("count")).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
|Red Hot Chili Pep...|   24|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [58]:
user_log.filter(user_log.page == "NextSong").\
groupby("sessionId").count().\
withColumnRenamed("count", "songcount").\
agg(F.round(F.avg("songcount"), 0).cast(IntegerType()).\
    alias("avgsc")).\
show()

+-----+
|avgsc|
+-----+
|    7|
+-----+

